In [2]:
!pip3 install scikit-learn
!pip3 install pandas

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [146]:
import pandas as pd 
input_file = "cardio_train.csv"
df = pd.read_csv(input_file, header=0)


In [169]:
import pandas as pd 
from sklearn.preprocessing import OneHotEncoder 

df_cat = df.select_dtypes(include='object')
encoder = OneHotEncoder(sparse=False, handle_unknown='error')
df_cat_encoded = encoder.fit_transform(df_cat)
categorical_columns = [f'{col}_{cat}' for i, col in enumerate(df_cat.columns) for cat in encoder.categories_[i]]
df_num = df.select_dtypes(exclude='object')
df_cat = pd.DataFrame(df_cat_encoded, columns=categorical_columns)
df = df_num.join(df_cat)
df

,id,age,height,weight,ap_hi,ap_lo,smoke,alco,active,cardio,gender_man,gender_women,above_above,above_normal,above_well above,gluc_above,gluc_normal,gluc_well above
0,0,18393,168,62.0,110,80,0,0,1,0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,1,20228,156,85.0,140,90,0,0,1,1,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
2,2,18857,165,64.0,130,70,0,0,0,1,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
3,3,17623,169,82.0,150,100,0,0,1,1,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,4,17474,156,56.0,100,60,0,0,0,0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,99993,19240,168,76.0,120,80,1,0,1,0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
69996,99995,22601,158,126.0,140,90,0,0,1,1,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
69997,99996,19066,183,105.0,180,90,0,1,0,1,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
69998,99998,22431,163,72.0,135,80,0,0,0,1,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0


In [172]:
Y = df[["cardio"]]
X = df.loc[:, df.columns != 'cardio']

In [173]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=1)
x_train = x_train.to_numpy()
y_train = y_train.to_numpy()
x_test = x_test.to_numpy()
y_test = y_test.to_numpy()
y_train = y_train.ravel()
y_test = y_test.ravel()

In [134]:
# start of naive bayes
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
k = 20
model.fit(x_train, y_train)
model.class_prior_

array([0.37362637, 0.62637363])

In [121]:
from sklearn.model_selection import KFold, GridSearchCV, StratifiedKFold

best_kf = 50
best_kf_value = 0 
best_st = 50 
best_st_value = 0

for k in range(50, 250, 50):
    kf = KFold(n_splits=k)
    gs = GridSearchCV(model, cv=kf, param_grid={}, return_train_score=True)
    gs.fit(x_train, y_train)
    if k == 50:
        print(gs.cv_results_)
    if gs.cv_results_['mean_test_score'][0] >= best_kf_value: 
        best_kf_value = gs.cv_results_['mean_test_score'][0]
        best_kf = k 
    
    stf = StratifiedKFold(n_splits=k)
    gs = GridSearchCV(model, cv=stf, param_grid={}, return_train_score=True)
    gs.fit(x_train, y_train)
    if gs.cv_results_['mean_test_score'][0] >= best_st_value: 
        best_st_value = gs.cv_results_['mean_test_score'][0]
        best_st = k
        
print(best_kf)
print(best_kf_value)
print(best_st)
print(best_st_value)

{'mean_fit_time': array([0.00122139]), 'std_fit_time': array([0.00038619]), 'mean_score_time': array([0.00030621]), 'std_score_time': array([7.71743906e-05]), 'params': [{}], 'split0_test_score': array([0.6]), 'split1_test_score': array([0.5]), 'split2_test_score': array([0.5]), 'split3_test_score': array([0.7]), 'split4_test_score': array([0.9]), 'split5_test_score': array([0.66666667]), 'split6_test_score': array([1.]), 'split7_test_score': array([0.66666667]), 'split8_test_score': array([0.55555556]), 'split9_test_score': array([0.44444444]), 'split10_test_score': array([0.77777778]), 'split11_test_score': array([0.77777778]), 'split12_test_score': array([0.33333333]), 'split13_test_score': array([0.66666667]), 'split14_test_score': array([0.33333333]), 'split15_test_score': array([0.66666667]), 'split16_test_score': array([0.77777778]), 'split17_test_score': array([0.55555556]), 'split18_test_score': array([0.77777778]), 'split19_test_score': array([0.22222222]), 'split20_test_scor

/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 170 members, which is less than n_splits=200.
  warnings.warn(


150
0.6272222222222221
100
0.63


In [100]:
from sklearn.model_selection import cross_val_score
best_kf_cv = KFold(n_splits=best_kf)
model = BernoulliNB()
result = cross_val_score(model, x_train, y_train, cv=best_kf_cv)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example 

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example 

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example 

In [102]:
print("Best k for k cross validation is: ", best_kf)
result.shape

Best k for k cross validation is:  150


(150,)

In [105]:
model.

BernoulliNB()